In [1]:
import os
import json
import pickle
import random
from collections import defaultdict, Counter

from indra.literature.adeft_tools import universal_extract_text
from indra.databases.hgnc_client import get_hgnc_name, get_hgnc_id

from adeft.discover import AdeftMiner
from adeft.gui import ground_with_gui
from adeft.modeling.label import AdeftLabeler
from adeft.modeling.classify import AdeftClassifier
from adeft.disambiguate import AdeftDisambiguator, load_disambiguator


from adeft_indra.ground.ground import AdeftGrounder
from adeft_indra.model_building.s3 import model_to_s3
from adeft_indra.model_building.escape import escape_filename
from adeft_indra.db.content import get_pmids_for_agent_text, get_pmids_for_entity, \
    get_plaintexts_for_pmids

In [2]:
adeft_grounder = AdeftGrounder()

In [3]:
shortforms = ['RAC']
model_name = ':'.join(sorted(escape_filename(shortform) for shortform in shortforms))
results_path = os.path.abspath(os.path.join('../..', 'results', model_name))

In [4]:
miners = dict()
all_texts = {}
for shortform in shortforms:
    pmids = get_pmids_for_agent_text(shortform)
    if len(pmids) > 10000:
        pmids = random.choices(pmids, k=10000)
    text_dict = get_plaintexts_for_pmids(pmids, contains=shortforms)
    text_dict = {pmid: text for pmid, text in text_dict.items() if len(text) > 5}
    miners[shortform] = AdeftMiner(shortform)
    miners[shortform].process_texts(text_dict.values())
    all_texts.update(text_dict)

longform_dict = {}
for shortform in shortforms:
    longforms = miners[shortform].get_longforms()
    longforms = [(longform, count, score) for longform, count, score in longforms
                 if count*score > 2]
    longform_dict[shortform] = longforms
    
combined_longforms = Counter()
for longform_rows in longform_dict.values():
    combined_longforms.update({longform: count for longform, count, score
                               in longform_rows})
grounding_map = {}
names = {}
for longform in combined_longforms:
    groundings = adeft_grounder.ground(longform)
    if groundings:
        grounding = groundings[0]['grounding']
        grounding_map[longform] = grounding
        names[grounding] = groundings[0]['name']
longforms, counts = zip(*combined_longforms.most_common())
pos_labels = []

In [5]:
list(zip(longforms, counts))

[('radial alveolar count', 25),
 ('raclopride', 24),
 ('ractopamine', 22),
 ('ribosome associated complex', 20),
 ('rabbit articular chondrocytes', 12),
 ('ractopamine hydrochloride', 12),
 ('relative area change', 10),
 ('regenerated amorphous cellulose', 9),
 ('renal artery calcium', 6),
 ('right auditory cortex', 4),
 ('ractopamine hcl', 4),
 ('regional anticoagulation with citrate', 3),
 ('ractopamine treatment', 3),
 ('relative c3 botulinum toxin substrate', 3)]

In [6]:
grounding_map, names, pos_labels = ground_with_gui(longforms, counts, 
                                                   grounding_map=grounding_map,
                                                   names=names, pos_labels=pos_labels, no_browser=True, port=8890)

INFO: [2020-09-25 14:47:31] indra.ontology.bio.ontology - Loading INDRA bio ontology from cache at /home/ubuntu/.indra/bio_ontology/1.2/bio_ontology.pkl


In [7]:
result = [grounding_map, names, pos_labels]

In [8]:
result

[{'benzaldehyde lyase': 'MESH:C059416',
  'betaine aldehyde': 'CHEBI:CHEBI:15710',
  'bile salt activity lipase': 'HGNC:1848',
  'bioartificial liver': 'MESH:D019164',
  'blood alcohol levels': 'ungrounded',
  'breath alcohol levels': 'ungrounded',
  'british anti lewisite': 'CHEBI:CHEBI:64198',
  'brochoalveolar lavage': 'MESH:D018893',
  'bronchalveolar lavage': 'MESH:D018893',
  'bronchial alveolar lavage': 'MESH:D018893',
  'bronchial lavage': 'MESH:D018893',
  'bronchio alveolar lavage': 'MESH:D018893',
  'bronchiolar lavage': 'MESH:D018893',
  'broncho alveolar lavage': 'MESH:D018893',
  'bronchoalveolar': 'MESH:D018893',
  'bronchoalveolar fluid': 'MESH:D018893',
  'bronchoalveolar larvage': 'MESH:D018893',
  'bronchoalveolar lavage': 'MESH:D018893'},
 {'MESH:C059416': 'benzaldehyde lyase',
  'CHEBI:CHEBI:15710': 'betaine aldehyde',
  'HGNC:1848': 'CEL',
  'MESH:D019164': 'Liver, Artificial',
  'CHEBI:CHEBI:64198': 'dimercaprol',
  'MESH:D018893': 'Bronchoalveolar Lavage'},
 ['H

In [66]:
grounding_map, names, pos_labels = ({'rabbit articular chondrocytes': 'ungrounded',
  'raclopride': 'MESH:D020891',
  'ractopamine': 'CHEBI:CHEBI:82644',
  'ractopamine hcl': 'CHEBI:CHEBI:82644',
  'ractopamine hydrochloride': 'CHEBI:CHEBI:82644',
  'radial alveolar count': 'radial alveolar count',
  'ras relative c3 botulinum toxin substrate': 'FPLX:RAC',
  'regenerated amorphous cellulose': 'ungrounded',
  'regional anticoagulation with citrate': 'ungrounded',
  'relative area change': 'ungrounded',
  'renal artery calcification': 'ungrounded',
  'renal artery calcium': 'ungrounded',
  'ribosome associated complex': 'ungrounded',
  'right auditory cortex': 'ungrounded'},
 {'MESH:D020891': 'Raclopride',
  'CHEBI:CHEBI:82644': 'ractopamine',
  'radial alveolar count': 'radial alveolar count',
  'FPLX:RAC': 'RAC'},
 ['CHEBI:CHEBI:82644', 'FPLX:RAC', 'MESH:D020891'])

In [67]:
excluded_longforms = []

In [68]:
grounding_dict = {shortform: {longform: grounding_map[longform] 
                              for longform, _, _ in longforms if longform in grounding_map
                              and longform not in excluded_longforms}
                  for shortform, longforms in longform_dict.items()}
result = [grounding_dict, names, pos_labels]

if not os.path.exists(results_path):
    os.mkdir(results_path)
with open(os.path.join(results_path, f'{model_name}_preliminary_grounding_info.json'), 'w') as f:
    json.dump(result, f)

In [69]:
additional_entities = {'HGNC:391': ['AKT1', ['RAC']]}

In [70]:
unambiguous_agent_texts = {}

In [71]:
labeler = AdeftLabeler(grounding_dict)
corpus = labeler.build_from_texts((text, pmid) for pmid, text in all_texts.items())
agent_text_pmid_map = defaultdict(list)
for text, label, id_ in corpus:
    agent_text_pmid_map[label].append(id_)

entity_pmid_map = {entity: set(get_pmids_for_entity(*entity.split(':', maxsplit=1),
                                                major_topic=True))for entity in additional_entities}

In [72]:
intersection1 = []
for entity1, pmids1 in entity_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection1.append((entity1, entity2, len(pmids1 & pmids2)))

In [73]:
intersection2 = []
for entity1, pmids1 in agent_text_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection2.append((entity1, entity2, len(set(pmids1) & pmids2)))

In [74]:
intersection1

[('HGNC:391', 'HGNC:391', 15527)]

In [75]:
intersection2

[('MESH:D020891', 'HGNC:391', 0),
 ('ungrounded', 'HGNC:391', 0),
 ('CHEBI:CHEBI:82644', 'HGNC:391', 0),
 ('radial alveolar count', 'HGNC:391', 0)]

In [76]:
all_used_pmids = set()
for entity, agent_texts in unambiguous_agent_texts.items():
    used_pmids = set()
    for agent_text in agent_texts[1]:
        pmids = set(get_pmids_for_agent_text(agent_text))
        new_pmids = list(pmids - all_texts.keys() - used_pmids)
        text_dict = get_plaintexts_for_pmids(new_pmids, contains=agent_texts)
        corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items() if len(text) >= 5])
        used_pmids.update(new_pmids)
    all_used_pmids.update(used_pmids)
        
for entity, pmids in entity_pmid_map.items():
    new_pmids = list(set(pmids) - all_texts.keys() - all_used_pmids)
    if len(new_pmids) > 10000:
        new_pmids = random.choices(new_pmids, k=10000)
    _, contains = additional_entities[entity]
    text_dict = get_plaintexts_for_pmids(new_pmids, contains=[])
    corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items() if len(text) >= 5])

In [77]:
names.update({key: value[0] for key, value in additional_entities.items()})
names.update({key: value[0] for key, value in unambiguous_agent_texts.items()})
pos_labels = list(set(pos_labels) | additional_entities.keys() |
                  unambiguous_agent_texts.keys())

In [78]:
%%capture

classifier = AdeftClassifier(shortforms, pos_labels=pos_labels, random_state=1729)
param_grid = {'C': [100.0], 'max_features': [10000]}
texts, labels, pmids = zip(*corpus)
classifier.cv(texts, labels, param_grid, cv=5, n_jobs=5)

INFO: [2020-10-28 21:42:13] /adeft/PP/adeft/adeft/modeling/classify.py - Beginning grid search in parameter space:
{'C': [100.0], 'max_features': [10000]}
INFO: [2020-10-28 21:43:59] /adeft/PP/adeft/adeft/modeling/classify.py - Best f1 score of 0.9954120553432111 found for parameter values:
{'logit__C': 100.0, 'tfidf__max_features': 10000}


In [79]:
classifier.stats

{'label_distribution': {'MESH:D020891': 19,
  'ungrounded': 39,
  'CHEBI:CHEBI:82644': 36,
  'radial alveolar count': 10,
  'HGNC:391': 7387},
 'f1': {'mean': 0.995412, 'std': 0.001429},
 'precision': {'mean': 0.994439, 'std': 0.001333},
 'recall': {'mean': 0.997178, 'std': 0.001302},
 'ungrounded': {'f1': {'mean': 0.670263, 'std': 0.103478},
  'pr': {'mean': 0.539286, 'std': 0.12289},
  'rc': {'mean': 0.92, 'std': 0.09798}},
 'radial alveolar count': {'f1': {'mean': 0.6, 'std': 0.326599},
  'pr': {'mean': 0.5, 'std': 0.316228},
  'rc': {'mean': 0.8, 'std': 0.4}},
 'CHEBI:CHEBI:82644': {'f1': {'mean': 0.753333, 'std': 0.100222},
  'pr': {'mean': 0.614286, 'std': 0.12454},
  'rc': {'mean': 1.0, 'std': 0.0}},
 'HGNC:391': {'f1': {'mean': 0.997097, 'std': 0.000694},
  'pr': {'mean': 0.999594, 'std': 0.000542},
  'rc': {'mean': 0.994614, 'std': 0.001271}},
 'MESH:D020891': {'f1': {'mean': 0.813333, 'std': 0.243676},
  'pr': {'mean': 0.8, 'std': 0.291548},
  'rc': {'mean': 0.92, 'std': 0.16

In [80]:
disamb = AdeftDisambiguator(classifier, grounding_dict, names)

In [81]:
disamb.dump(model_name, results_path)

In [82]:
print(disamb.info())

Disambiguation model for RAC

Produces the disambiguations:
	AKT1*	HGNC:391
	RAC*	FPLX:RAC
	Raclopride*	MESH:D020891
	ractopamine*	CHEBI:CHEBI:82644
	radial alveolar count	radial alveolar count

Class level metrics:
--------------------
Grounding            	Count	F1     
                 AKT1*	7387	 0.9971
           Ungrounded	  39	0.67026
          ractopamine*	  36	0.75333
           Raclopride*	  19	0.81333
radial alveolar count	  10	    0.6

Weighted Metrics:
-----------------
	F1 score:	0.99541
	Precision:	0.99444
	Recall:		0.99718

* Positive labels
See Docstring for explanation



In [63]:
model_to_s3(disamb)

In [49]:
preds = [disamb.disambiguate(text) for text in all_texts.values()]

In [54]:
texts = [text for text, pred in zip(all_texts.values(), preds) if pred[0] == 'HGNC:391']

In [65]:
texts[30]

'CDC42/RAC-independent activation of PAK4–6\nAlthough PAKs are bind to Cdc42 and Rac, a number of stimuli which could operate independently of these GTPases may also lie upstream of the PAKs. In MDCK epithelial cells, for example, PAK4 is activated by hepatocyte growth factor (HGF) via a signaling pathway that requires PI3 kinase. HGF also causes PAK4 to localize to the cell periphery, and PAK4 is in turn thought to contribute to HGF induced changes in cytoskeletal organization and cell adhesion.36 PAK4 was also shown to interact with the cytoplasmic domain of the keratinocyte growth factor (KGF) receptor in a transformed kidney cell line, and may have a role in KGFR mediated cell survival pathways.37 PAK6 has been linked to androgen receptor signaling,11,13,14 by a mechanism not known to be linked to CDC42/RAC. PAK6 was also shown to be activated by MAP kinase kinase 6 (MKK6) and by the p38 MAP Kinase.38 These data suggest that multiple types of signaling pathways may converge upon PA